# <center>Workflow for on-line GC and HPLC analysis in flow chemistry</center>


---
## Section 0: Imports, Paths, and Logging
---

In this section all the necessary python packages are imported, the path to this notebook and the logger for this notebook is set up.

In [1]:
# Activate autoreload to keep on track with changing modules #
%reload_ext autoreload
%autoreload 2

# Import standard libraries #
import numpy as np
import logging
import json
from pathlib import Path

# Import tools for data processing and analysis #
from datamodel.tools import initialize_dataset
from datamodel.tools import reading_raw_data_widget
from datamodel.tools import analyzing_raw_data_widget


In [2]:
# Define paths for loggin output #
root                = Path.cwd()
logging_config_path = root / "datamodel/tools/logger_config.json"

# Read in logger specs and configurate logger (set name to current notebook) #
with open(logging_config_path) as logging_config_json: logging.config.dictConfig( json.load( logging_config_json ) )
logger = logging.getLogger(__name__)

# Set the level of thid-party logger to avoid dumping too much information #
for logger_ in ['markdown_it', 'h5py', 'numexpr', 'git']: logging.getLogger(logger_).setLevel('WARNING')

---
## Section 1: Dataset and raw data
---
In this section the dataset as well as the to analyze raw data is choosen

In [7]:
git_path = 'https://github.com/FAIRChemistry/datamodel_b07_tc.git'
branch   = 'samir_develop'

id = initialize_dataset()
id.write_dataset(root, git_path, branch)

In [5]:
## Definition of basic meta data of the project ##

id.title.value        = 'Electrocatalytic CO2-reduction on carbon'
id.description.value  = 'The aim of this project is to blablabla'

# List with authors and their affiliation #
id.authors.value      = 'Richard Schömig, Maximilian Schmidt' 
id.affiliations.value = 'University of Stuttgart, University of Stuttgart'
id.dataset_text.value = "test"

In [13]:
## Search for dataset and raw data ##

rrdw = reading_raw_data_widget()
rrdw.choose_data(root)

In [7]:
#str(root) = "c:/Users/darouich/OneDrive/Dokumente/"

e_chem = str(root)+'/data/Rohdaten/01_EChem/CAD14-Cu@AB/GSTATIC.DTA'
mfm    = str(root)+'/data/Rohdaten/03_MFM/CAD14-Cu@AB/Bench-2h-GSS_CAD14-Cu@AB_200_50c_24h_truncated.csv'
gc     = [str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0103.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0103.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0104.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0104.D/REPORT01.CSV']
calib  = str(root)+'/data/calibration/calibration.json'
correc = str(root)+'/data/correction_factors/correction_factors.json'
farada = str(root)+'/data/faraday_coefficients/faraday_coefficients.json'

rrdw.Echem_files.value = [e_chem]
rrdw.MFM_files.value   = [mfm]
rrdw.GC_files.value    = gc
rrdw.calib_files.value = [calib]
rrdw.correction_files.value = [correc]
rrdw.faraday_files.value    = [farada]

---
## Section 2: Analysis of data
---
In this section the raw data of the above choosen dataset is analyzed (if you change the dataset above, then reexecute this cell)

In [27]:
# Provide a typical retention time dictionary to pre assign retention times 

typical_retention_time = {"Hydrogen": 1.7, "Carbon dioxide": 3.0, "Carbon monoxide": 13.6, 
                          "Methane": 3.6, "Ethene": 6.0, "Ethane": 7.1}

ardw = analyzing_raw_data_widget()
ardw.choose_experiment( datamodel = rrdw.datamodel, 
                        dataset_path = rrdw.dataset_dropdown.value, 
                        typical_retention_time = typical_retention_time)


Starting the postprocessing


!!! Warning: Volume fractions doesn't add up to 1.0 !!!

Faraday effiencies of GC measurement n°0
                Faraday_efficiency [%]
Carbon dioxide              245.965934
Carbon monoxide               5.898531
Ethene                         12.1804
Hydrogen                     14.528083
Methane                        7.80829 

Faraday effiencies of GC measurement n°1
                Faraday_efficiency [%]
Carbon dioxide              250.311538
Carbon monoxide               7.082942
Ethene                       10.766099
Hydrogen                     19.263657
Methane                       8.853112 

Faraday effiencies of GC measurement n°2
                Faraday_efficiency [%]
Carbon dioxide              262.134414
Carbon monoxide               6.829024
Ethene                        9.418884
Hydrogen                     18.747449
Methane                       9.164263 


Mean Faraday efficency over all GC measurements
                Faraday_efficien

---
## Section 3: Upload of data to DaRUS
---
In this section the dataset containing the processed as well as the raw data, is uploaded to DaRUS

In [61]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from pyDaRUS import Citation
from pyDaRUS import Dataset as DaRUS_dataset
from pyDaRUS.metadatablocks.citation import SubjectEnum, IdentifierScheme, RelatedPublication, TopicClassification, Keyword
from datetime import datetime

import os
import ipywidgets as widgets
import pandas as pd
from pathlib import Path
from IPython.display import display, clear_output

# Import modified sdRDM objects #
from sdRDM import DataModel

# Import general tools and objects of this datamodel #

# Objects #
from datamodel.core import Experiment
from datamodel.core import MeasurementType
from datamodel.core import Quantity

# Tools #
from datamodel.tools.auxiliary import Librarian

In [7]:
class DaRUS_upload:

    def upload_to_DaRUS(self,_):

        #### Initialize and write DaRUS dataset ####

        DaRUS_dataset = DaRUS_dataset()

        ## Get citation metadata from the general information object of the provided dataset ##

        citation      = Citation()



        ## **{k:author.__dict__[k] for k in author.__dict__.keys() if k!="id"}

        # Extract general information from the provided dataset
        citation.add_project( name = self.dataset.general_information.project, level=1 )
        
        citation.title = self.dataset.general_information.title
        
        for author in self.dataset.general_information.authors: citation.add_author( **author.__dict__ ) 

        citation.add_contact( **self.dataset.general_information.contact.__dict__ )

        citation.subject      = self.dataset.general_information.subject

        citation.depositor    = self.depositor_text.value.strip()
        citation.deposit_date = datetime.now().date().strftime("%Y-%m-%d")
        
        citation.add_grant_information( grant_agency="DFG", grant_number="358283783 - SFB 1333")

        citation.language     = "English"

        citation.add_related_publication( **self.dataset.general_information.related_publication.__dict__ )
        
        for classification in self.dataset.general_information.topic_classifications: citation.add_topic_classification( **classification.__dict__ )

        for keyword in self.dataset.general_information.keywords: citation.add_keyword( **keyword.__dict__ )

        DaRUS_dataset.add_metadatablock(citation)
        
        ## Get Prcoess metadata from the general information object of the provided dataset ##



        ## Add files and directories ##

        DaRUS_dataset.add_directory("Examples/dataset_upload/")
        DaRUS_dataset.add_file(dv_path="my.file", local_path="my.file")

        ## Upload ##

        DaRUS_dataset.upload(dataverse_name = self.dataverse_dropdown.value,
                             DATAVERSE_URL  = "https://darus.uni-stuttgart.de",
                             API_TOKEN      = self.api_token_text.value)

    def upload(self,datamodel,dataset_path,dataverse_list):
        
        # Common variables

        self.dataset_path           = dataset_path
        self.dataset, self.lib      = datamodel
        
        self.dataverse_dropdown     = widgets.Dropdown(options= dataverse_list,
                                                    description="Choose dataverse",
                                                    layout=widgets.Layout(width='auto'),
                                                    style={'description_width': 'auto'})
        
        self.depositor_text         = widgets.Text (description="Depositor:",
                                                    placeholder="Name of the person uploading this dataset (e.g.: Max Mustermann)",
                                                    layout=widgets.Layout(width='auto'),
                                                    style={'description_width': 'auto'})
        
        self.api_token_text         = widgets.Text (description="API token:",
                                                    placeholder="Provide personal API token from DaRUS (e.g.: xxx-xxx-xxx-xxx-xxx)",
                                                    layout=widgets.Layout(width='auto'),
                                                    style={'description_width': 'auto'})

        
        
        self.button_upload          = widgets.Button(description='Upload dataset to DaRUS',
                                                     layout=widgets.Layout(width="30%"),
                                                     style={"button_color": 'lightblue'})
        
        # Handle button
        self.button_upload.on_click(self.upload_to_DaRUS)

        
        display(self.subject_dropdown)

In [8]:
test = DaRUS_upload()

test.upload()

Dropdown(description='Choose datamodel', layout=Layout(width='auto'), options=('Agricultural Sciences', 'Arts …

In [ ]:
sfb1333_dataverse_list = [ "sfb1333-hansen-gross" ]
api_token = "4afecd82-c92d-4935-b786-2225af43531e"

In [2]:


subject_list = [ subject.value for subject in SubjectEnum ]

# Initialize the dataset and the metadatablock
dataset  = Dataset()
citation = Citation()

citation.title = "My Title"

citation.add_description(text="test")
citation.add_author(name="Samir", affiliation="ITT")
citation.add_contact(name="Samir", email="samir.darouich@itt.uni-stuttgart.de")

citation.subject = [SubjectEnum.engineering]


citation.depositor = "max "
citation.deposit_date = "2023"


dataset.add_metadatablock(citation)

dataset.add_file(dv_path="my.file", local_path="my.file")



Attribute project not valid for import (dv_up).
Dataset with pid 'doi:10.18419/darus-3806' created.
Uploading data files: 100%|██████████| 1/1 [00:01<00:00,  1.02s/it]
https://darus.uni-stuttgart.de/dataset.xhtml?persistentId=doi:10.18419/darus-3806


'doi:10.18419/darus-3806'

In [3]:
subject_list

['Agricultural Sciences',
 'Arts and Humanities',
 'Astronomy and Astrophysics',
 'Business and Management',
 'Chemistry',
 'Computer and Information Science',
 'Earth and Environmental Sciences',
 'Engineering',
 'Law',
 'Mathematical Sciences',
 'Medicine, Health and Life Sciences',
 'Physics',
 'Social Sciences',
 'Other']